## <RNN 기반 인랑 판독기 구현>에 사용되는 데이터를 스크랩핑하기 위한 소스코드

BeautifulSoup와 Selenium을 이용하여 스크랩핑을 진행했다. <br>

비밀 메세지들은 제외하고, 전체 공개인 메세지만을 수집하였고, 메세지의 내용, 캐릭터의 이름, 게임에서의 역할을 함께 저장하였다. 1일째~마지막날까지 페이지를 이동하며 데이터를 수집하였고, 실제 게임과는 관련이 없는 프롤로그와 에필로그 페이지는 제외했다. <br>

총 23개의 마을에서 스크랩핑을 진행하여 각각 파일로 저장하였다.

# 준비

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
import pandas as pd

import requests
from bs4 import BeautifulSoup


import time
import datetime

import warnings
warnings.filterwarnings('ignore')

import gzip

페이지가 아코디언 방식으로 내용이 접혀있어 스크랩핑을 위해서는 브라우저 조작이 필요하다.<br>
따라서, 셀레니움을 이용할 수 있도록 크롬 드라이버와 관련 라이브러리의 설치가 필요.

In [ ]:
## 코드 출처: https://pgh268400.tistory.com/286

# 최근 우분투 업데이트에서 크롬 드라이버 설치를 snap을 이용해서만 하도록 바뀜
# 고로 snap 없이 설치하는 아래 우회 코드로 변경
# 출처 : https://colab.research.google.com/drive/1cbEvuZOhkouYLda3RqiwtbM-o9hxGLyC
# 출처2 : https://stackoverflow.com/questions/75155063/selenium-use-chrome-on-colab-got-unexpectedly-exited

%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.SUASvNh7JO/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.PnJ7YGfbWH/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.1n76Cr9Rbc/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 https://cloud.r-project.org/bin/l

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


In [ ]:
# 파일을 저장할 위치 설정

file_path =  "/content/gdrive/My Drive/Colab Notebooks/scrapData/"

## ■ 사이트에서 데이터를 수집

데이터 출처 사이트: http://werewolf.co.kr/bbs/zboard.php?id=werewolf





In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'}
title = ''
def scraper(gameNum):
    global title
    log_df = pd.DataFrame(columns=("Name", "Role", "Message"))
    wolf_url = 'http://werewolf.co.kr/bbs/view.php?id=werewolf&no={0}'.format(str(gameNum))

    req = requests.get(wolf_url)

    # cont = req.content 부분에서 에러 발생
    # WARNING:bs4.dammit:Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
    # Content-Encoding: gzip 문구를 보고 검색을 통해 본문이 압축된 상태라는 것을 알게됨

    cont = gzip.decompress(req.content)

    soup = BeautifulSoup(cont,'html.parser',from_encoding='EUR-KR')

    # 마을 이름을 파일명으로 사용할 것이므로 global 변수 title에 저장
    title = soup.select('#viewStateAll .content')[0].text

    # viewDay의 마지막 날은 게임이 끝나고 난 후 플레이어들끼리 후일담을 나누는 에필로그이므로 제외
    lastDay = soup.select('div.viewDay a')[-2].text[0]
    index = 0
    viewDay = 1

    while viewDay <= int(lastDay):
        # viewMode=all로 설정해야 각 플레이어의 역할이 보임
        # 단, 이 경우 개인 메모와 인랑 비밀 메세지를 비롯한 모든 로그가 함께 열람되므로
        # class를 지정하여 공개 메세지만 스크랩핑하여야 함

        url = '{0}&viewDay={1}&viewChar=&viewMode=all'.format(wolf_url, str(viewDay))

        # 주소를 옮길 때마다 드라이버를 초기화해주어야 함
        driver = webdriver.Chrome(options=options)
        driver.implicitly_wait(1)
        driver.get(url)
        driver.implicitly_wait(1)
        element = driver.find_element(By.ID, "buttonOpenCommentPagesAll")
        driver.execute_script("arguments[0].click()",element)

        # 모든 로그를 펼치는 시간을 기다림. 에러가 뜬다면 보통 이 단계에서 로딩이 완료되기 전 접근을 시도하는 것이므로 시간을 늘릴 것.
        # 페이지 자체의 로딩이 오래 걸릴 수 있으니 위의 implicitly_wait의 시간을 늘리는 것도 시도할 수 있다.
        time.sleep(30)

        html = driver.page_source

        print('▶ 수집 중인 페이지 = ', url)

        soup = BeautifulSoup(html,'html.parser',from_encoding='EUR-KR')

        for urls in soup.select(".comment.normal"):
            try:
                name = urls.select('span.c_name>label')[0].text
            except:
                name = urls.select('span.c_Name>label')[0].text

            role = urls.select('span.playerInfo')[0].text.split(' / ')[1]
            message = urls.select('.message')[0].text.replace('\n',' ').replace('\t',' ').replace('\r',' ')

            log_df.loc[index] = [name, role, message]
            index += 1

        viewDay += 1

    print()
    print('Completed! 🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞 ')

    return log_df




총 23개의 마을을 스크랩핑했다. <br>
각 마을의 게임 코드는 다음과 같으며, 셀레니움이 페이지가 로딩되기 전 클릭을 시도하는 등 인터넷 연결 상태에 따라 오류가 생길 수 있다.<br>

['2486', '3326', '4915', '2765', '3130', '2463', '2823', '1762', '1628', '1362', '1454', '1497', '1537', '6553', '6718', '4119', '4591', '6149', '5878', '5777', '5592', '4154', '4110']

In [ ]:
for gameCode in ['5592','4154','4110']:
    log_df = scraper(gameCode)
    file_name = title + ".csv"
    file_full_path = file_path + file_name
    log_df.to_csv(file_full_path, index = False, encoding="utf-8-sig")


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 469, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/response.py", line 91, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'P3P : CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"\r\nSet-Cookie: PHPSESSID=a55f5e11292d678966673034dd0db52d; path=/\r\nContent-Encoding: gzip\r\n\r\n'


▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=5592&viewDay=1&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=5592&viewDay=2&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=5592&viewDay=3&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=5592&viewDay=4&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=5592&viewDay=5&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=5592&viewDay=6&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=5592&viewDay=7&viewChar=&viewMode=all

Completed! 🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞 


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 469, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/response.py", line 91, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'P3P : CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"\r\nSet-Cookie: PHPSESSID=707fb2ac2b5c41984f778403e6e85845; path=/\r\nContent-Encoding: gzip\r\n\r\n'


▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4154&viewDay=1&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4154&viewDay=2&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4154&viewDay=3&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4154&viewDay=4&viewChar=&viewMode=all

Completed! 🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞 


Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 469, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/response.py", line 91, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'P3P : CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"\r\nSet-Cookie: PHPSESSID=ae25d5037ccbcc4400682afbad4132c4; path=/\r\nContent-Encoding: gzip\r\n\r\n'


▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4110&viewDay=1&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4110&viewDay=2&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4110&viewDay=3&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=4110&viewDay=4&viewChar=&viewMode=all

Completed! 🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞 


## 단일 마을을 스크랩핑하거나 파일을 열어 내용을 확인하고자 하는 경우

In [ ]:
start = time.time()    # 시작 시간 기록

gameNum = '3130'
log_df = scraper(gameNum)


processing_time = time.time() - start    # 경과 시간 계산
times = str(datetime.timedelta(seconds = processing_time))

print()
print("◎ 수행시간 = ", times)

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 469, in _make_request
    assert_header_parsing(httplib_response.msg)
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/response.py", line 91, in assert_header_parsing
    raise HeaderParsingError(defects=defects, unparsed_data=unparsed_data)
urllib3.exceptions.HeaderParsingError: [MissingHeaderBodySeparatorDefect()], unparsed data: 'P3P : CP="ALL CURa ADMa DEVa TAIa OUR BUS IND PHY ONL UNI PUR FIN COM NAV INT DEM CNT STA POL HEA PRE LOC OTC"\r\nSet-Cookie: PHPSESSID=452bd790159f9fb3a6b261029192da14; path=/\r\nContent-Encoding: gzip\r\n\r\n'


▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=3130&viewDay=1&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=3130&viewDay=2&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=3130&viewDay=3&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=3130&viewDay=4&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=3130&viewDay=5&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=3130&viewDay=6&viewChar=&viewMode=all
▶ 수집 중인 페이지 =  http://werewolf.co.kr/bbs/view.php?id=werewolf&no=3130&viewDay=7&viewChar=&viewMode=all

Completed! 🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞🌞 

◎ 수행시간 =  0:03:59.117727


In [ ]:
log_df[:10]

,Name,Role,Message
0,NPC,마을사람,초심을 잃지 마시길..
1,이클립스,마을사람,ㄴㅁㄹㅇㄹㅇㄴㄹ 자러감
2,드라큐라,마을사람,점쟁이들 나오셔들.
3,인큐버스,마을사람,시작했군요.
4,드라큐라,마을사람,아무도 없... 이클립스 한명뿐?
5,이클립스,마을사람,양심상 점코는 보고갈랬는데 넘 느리다
6,안식을 주는 자,인랑,안녕하세요? 여러분께 안식과 평화를 가져다드릴 안식을 주는 자 입니다. 직업 확인했...
7,드라큐라,마을사람,점쟁이 나오라긔... 나도 자러 가야 해!
8,이클립스,마을사람,몰라 나 잘래 피곤해 드라큐라랑 인큐버스도 좋은꿈꿔
9,오크 베이비,초능력자,ㅂ ㅔ 이비 와쪄염 뿌우 ~'ㅅ' 근데여 님들 제가 마마몬 태클을 극심하게 뚫고 ...


In [ ]:
log_df.shape

(491, 3)

In [ ]:
log_df.head(20)

In [ ]:
log_df.tail(20)

In [ ]:
# 수집한 데이터를 저장하는 경로 설정

file_name = title + ".csv"
file_full_path = file_path + file_name


In [ ]:
# 수집한 데이터를 csv 파일로 저장

log_df.to_csv(file_full_path, index = False, encoding="utf-8-sig")

In [ ]:
# 저장된 데이터 파일 확인

report = pd.read_csv(file_full_path)

report.shape


(504, 3)

In [ ]:

report.head(10)


In [ ]:

report.tail(10)
